
    
# Converting a TensorFlow* Model in Linux*

## Pre-Requisites:


    
### Install Tensorflow 1.12 version

Run the following cells to install tensorflow 1.12 version

In [1]:
pip install --upgrade pip

  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/00/b6/9cfa56b4081ad13874b0c6f96af8ce16cfbc1cb06bedf8e9164ce5551ec1/pip-19.3.1-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

In [2]:
pip install --user tensorflow==1.12

  Using cached https://files.pythonhosted.org/packages/22/cc/ca70b78087015d21c5f3f93694107f34ebccb3be9624385a911d4b52ecef/tensorflow-1.12.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/07/53/8d32ce9471c18f8d99028b7cef2e5b39ea8765bd7ef250ca05b490880971/tensorboard-1.12.2-py3-none-any.whl
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

Create new temporary directory by running the following cell

In [3]:
! export TMPDIR=/tmp/$USER; mkdir -p $TMPDIR; python -c "import tempfile; print(tempfile.gettempdir())"

/tmp/u30239


In [4]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1 -o models

################|| Downloading models ||################

========== Downloading models/public/squeezenet1.1/squeezenet1.1.prototxt
... 100%, 9 KB, 27710 KB/s, 0 seconds passed

========== Downloading models/public/squeezenet1.1/squeezenet1.1.caffemodel
... 100%, 4834 KB, 27654 KB/s, 0 seconds passed

################|| Post-processing ||################

========== Replacing text in models/public/squeezenet1.1/squeezenet1.1.prototxt


### Download Models from TensorFlow* -slim library

There are a number of pre-trained public models in the TensorFlow*-slim repository. The models are distributed as Python scripts and checkpoint files. First of all download repository with models. Note: This is done in home directory(~)

In [5]:
! cd && git clone https://github.com/tensorflow/models/ && cd models/research/slim

fatal: destination path 'models' already exists and is not an empty directory.



    
### Export Inference Graph and download checkpoint file

Export inference graph for one of the available models using the following command (Inception V1 in this example):

In [7]:
! python3 -W ignore  $HOME/models/research/slim/export_inference_graph.py --alsologtostderr --model_name=inception_v1 --output_file=/tmp/$USER/inception_v1_inf_graph.pb  

INFO:tensorflow:Scale of 0 disables regularizer.



    
The script creates inference graph file with name “inception_v1_inf_graph.pb” in the /tmp direcory.

### Download archive with checkpoint file (Inception V1 in this example):

In [8]:
%%writefile run.sh
export CHECKPOINT_DIR=/tmp/$USER/checkpoints

mkdir $CHECKPOINT_DIR

wget http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz

tar -xvf inception_v1_2016_08_28.tar.gz

mv inception_v1.ckpt $CHECKPOINT_DIR

rm inception_v1_2016_08_28.tar.gz

Overwriting run.sh


In [9]:
%%bash
sh run.sh

inception_v1.ckpt


--2019-11-19 04:49:22--  http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.6.176, 2607:f8b0:4000:802::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.6.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24642554 (24M) [application/x-tar]
Saving to: ‘inception_v1_2016_08_28.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  576K 42s
    50K .......... .......... .......... .......... ..........  0% 1.15M 31s
   100K .......... .......... .......... .......... ..........  0% 2.04M 24s
   150K .......... .......... .......... .......... ..........  0% 2.43M 21s
   200K .......... .......... .......... .......... ..........  1% 3.87M 18s
   250K .......... .......... .......... .......... ..........  1% 4.35M 16s
   300K .......... .......... .......... .......... ..........  1% 5.59M 14s
   350K .......... .......... ...


    
### Freeze the model

The last step is to freeze the graph. To do this you need to know the output node of the model you are planning to freeze. This information is found by running the summarize_graph.

### Summarize Graph

Go to ~/models/research/slim/ directory and run summarize_graph.py script.

In [10]:
! cd $HOME/models/research/slim/ && python3 -W ignore  /opt/intel/openvino/deployment_tools/model_optimizer/mo/utils/summarize_graph.py --input_model=/tmp/$USER/inception_v1_inf_graph.pb 


1 input(s) detected:
Name: input, type: float32, shape: (-1,224,224,3)
1 output(s) detected:
InceptionV1/Logits/Predictions/Reshape_1



    
The output layer/node name should be in the last line of text on the console and should look like:

1 input(s) detected: Name: input, type: float32, shape: (-1,224,224,3) 1 output(s) detected: InceptionV1/Logits/Predictions/Reshape_1

### Freeze the graph for Intel® Distribution of OpenVINO™ toolkit

The script generates inception_v1_frozen.pb file with the frozen model in the directory you are currently in.

In [11]:
! python3 -W ignore /usr/local/lib/python3.6/dist-packages/tensorflow/python/tools/freeze_graph.py --input_graph /tmp/$USER/inception_v1_inf_graph.pb --input_binary --input_checkpoint /tmp/$USER/checkpoints/inception_v1.ckpt --output_node_names InceptionV1/Logits/Predictions/Reshape_1 --output_graph inception_v1_frozen.pb

2019-11-19 04:49:50.158279: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA



    
You might get warning message "Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA" while executing above command, ignore that.
## Convert Frozen Tensorflow* model to IR using Model Optimizer

Assuming you are in the ~/models/research/slim/ directory

In [12]:
! python3 -W ignore  /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model inception_v1_frozen.pb --input_shape [1,224,224,3] --mean_values [1024,1024,1024] --scale_values [128,128,128]

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/object-detection/devcloud/cpp/inception_v1_frozen.pb
	- Path for generated IR: 	/home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/object-detection/devcloud/cpp/.
	- IR output name: 	inception_v1_frozen
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,224,224,3]
	- Mean values: 	[1024,1024,1024]
	- Scale values: 	[128,128,128]
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared l


    
This should produce “inception_v1_frozen.xml” and “inception_v1_frozen.bin” file. The xml file contains the topology information of the model and the bin file contains the model’s weights and biases. These two files are expected when using the inference engine and so make note of the path.

## Run Classification Sample

The classification sample will showcase the Intel® Distribution of OpenVINO™ toolkit inference engine using TensorFlow model Inception_v1_frozen IR files (.xml & .bin) and an input image of a car to classify. The classification collateral is defined as the input image car_1.bmp, the Inception_v1_frozen IR files (.xml & .bin), and the labels file inception_v1_frozen.labels. Create a new directory that will hold the classification sample app and all needed components to run the classification sample Note: The following steps should be followed and are assuming you are following the preceding steps. You should be in the home directory.

In [13]:
!/opt/intel/openvino/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


In [14]:
! cp /opt/intel/openvino/deployment_tools/demo/squeezenet1.1.labels ./inception_v1_frozen.labels >/dev/null 2>&1


    
#### Run Application

Note: To see all the flags that the sample takes as input, run the below cell

In [15]:
! ~/inference_engine_samples_build/intel64/Release/classification_sample -h

[ INFO ] InferenceEngine: 
	API version ............ 2.1
	Build .................. custom_releases/2019/R3_cb6cad9663aea3d282e0e8b3e0bf359df665d5d0
	Description ....... API

classification_sample [OPTION]
Options:

    -h                      Print a usage message.
    -i "<path>"             Required. Path to a folder with images or path to an image files: a .ubyte file for LeNetand a .bmp file for the other networks.
    -m "<path>"             Required. Path to an .xml file with a trained model.
      -l "<absolute_path>"    Required for CPU custom layers. Absolute path to a shared library with the kernels implementations.
          Or
      -c "<absolute_path>"    Required for GPU custom kernels. Absolute path to the .xml file with the kernels descriptions.
    -pp "<path>"            Path to a plugin folder.
    -d "<device>"           Specify the target device to infer on; CPU, GPU, FPGA, HDDL or MYRIAD is acceptable. Sample will look for a suitable plugin for device specified (C

We will use the [car_1.png](car_1.png) image below to capture attributes 

In [16]:
! ~/inference_engine_samples_build/intel64/Release/classification_sample -i car_1.png  -m inception_v1_frozen.xml

[ INFO ] InferenceEngine: 
	API version ............ 2.1
	Build .................. custom_releases/2019/R3_cb6cad9663aea3d282e0e8b3e0bf359df665d5d0
	Description ....... API
[ INFO ] Parsing input parameters
[ INFO ] Files were added: 1
[ INFO ]     car_1.png
[ INFO ] Loading plugin

	API version ............ 2.1
	Build .................. 30677
	Description ....... MKLDNNPlugin
[ INFO ] Loading network files:
	inception_v1_frozen.xml
	inception_v1_frozen.bin
[ INFO ] Preparing input blobs
[ ERROR ] Data is empty!




Expected Output:


 
<img src="classification_output.png"> 


### Clean the output files before closing the lab
- point to the kernel in the header and click on *Restart & Clear Output* to clean the outputs of this page.

In [18]:
%%writefile clean_tf.sh
rm -rf incept*

Overwriting clean_tf.sh


In [20]:
! clean_tf.sh

/bin/sh: 1: clean_tf.sh: not found
